In [ ]:
from pathlib import Path
import time

import essentia
import essentia.standard as es
import librosa
import librosa.display
# IMPORTANT: since TCN is not available on pip version of madmom
# you have to build the library from source
import madmom
import matplotlib.pyplot as plt
import mirdata
import mir_eval
import numpy as np
import pandas as pd

import src.baseline as baseline
import src.utils as utils

In [ ]:
base_path = Path.cwd()

experiments_path = base_path / "results" / "baselines"
# output_path = base_path / "experiments_results" / "beat_trackers_baseline"
candombe_path = base_path.parent.parent / "datasets" / "candombe"

Important dataset information:
* Candombe:
    * sampling rate: 44100 Hz
    * precision: 16-bit
    * total audios: 36

In [ ]:
gtzan = mirdata.initialize('gtzan_genre', version='mini')
# gtzan.download()

In [ ]:
SR = 44100

In [ ]:
# helper dicts
baselines = {
    "librosa": baseline.librosa_beats,
    "madmom_rnn": baseline.madmom_rnn_beats,
    "madmom_tcn": baseline.madmom_tcn_beats,
    "essentia": baseline.essentia_beats
}

times = {
    "librosa": {},
    "madmom_rnn": {},
    "madmom_tcn": {},
    "essentia": {}
}

In [ ]:
# candombe
audiofiles = candombe_path.rglob("*.wav")

for audio in audiofiles:
    x, _ = librosa.load(audio, mono=True, sr=SR)
    
    for key, val in baselines.items():
        file_npz = experiments_path / key / audio.stem
        
        # if file exists, do nothing
        if not file_npz.is_file():
            start = time.perf_counter()
            beats = val(x)
            end = time.perf_counter()
            
            times[key][audio.stem] = end - start
            
            # is it better to create everything before looping?
            utils.create_folder(file_npz.parent)

            np.savez(file_npz, estimated=beats)

In [ ]:
# gtzan
for track_name in gtzan.track_ids:
    x, _ = librosa.load(gtzan.track(track_name).audio_path, mono=True, sr=SR)
    
    for key, val in baselines.items():
        file_npz = experiments_path / key / track_name
                
        # if file exists, do nothing
        if not file_npz.is_file():
            start = time.perf_counter()
            beats = val(x)
            end = time.perf_counter()

            times[key][track_name] = end - start

            # is it better to create everything before looping?
            utils.create_folder(file_npz.parent)

            np.savez(file_npz, estimated=beats)

# output examples 

datasets:

| track_id | dataset |
| --- | --- |
| track_1 | candombe | 
| track_2 | gtzan |


beats:

| track_id | dataset | reference | librosa_estimate | essentia_estimate | madmom_rnn_estimate | madmom_tcn_estimate |
| --- | --- | --- | --- | --- | --- | --- | 
| track_1 | gtzan | np.array([...]) | np.array([...]) | np.array([...]) | np.array([...]) | np.array([...]) | 

performance:

| track_id | dataset | librosa_time | essentia_time | madmom_rnn_time | madmom_tcn_time |
| --- | --- | --- | --- | --- | --- | 
| track_1 | gtzan | 0.99 | 0.98 | 0.87 | 0.99 | 1.2 | 


metrics:

| track_id | dataset | librosa_fmeasure | essentia_fmeasure | madmom_rnn_fmeasure | madmom_tcn_fmeasure |
| --- | --- | --- | --- | --- | --- | 
| track_1 | gtzan | 0.99 | 0.98 | 0.87 | 0.99 | 1.2 | 



In [ ]:
# build dataset dataframe
datasets = {}

for i in gtzan.track_ids:
    datasets[i] = f"gtzan.{i.split('.')[0]}"

for i in candombe_path.rglob("*.wav"):
    datasets[i.stem] = "candombe"

dataset_df = pd.DataFrame.from_dict(datasets, orient="index", columns=["dataset"])
dataset_df.index.name = "track_id"

dataset_df.to_csv(experiments_path / "experiment_data.csv")

In [ ]:
# build time dataframe
times_df = pd.DataFrame(times)
times_df.index.name = "track_id"

times_df.to_csv(experiments_path / "processing_time.csv")

In [ ]:
times_df = pd.read_csv(experiments_path / "processing_time.csv")
times_df = times_df.set_index("track_id")

In [ ]:
times_df.median()

In [ ]:
dataset_df = pd.read_csv(experiments_path / "experiment_data.csv")
dataset_df = dataset_df.set_index("track_id")

In [ ]:
# TODO: document the default parameter for those algorithms
# TODO: add more baselines? (e.g librosa with multi-channel?)
# TODO: evaluate baselines against ground_truth value 
# TODO: parallelize experiments run 

In [ ]:
# we have to group information in a dataframe or anything like this to 
# better analyze

In [ ]:
# we're not worried about downbeat estimation, so let's first just save our beats

# candombe
csvfiles = candombe_path.rglob("*.csv")
reference = {}

for file in csvfiles:
    x_df = pd.read_csv(file, names=["timestamp", "beat"])
    
    reference[file.stem] = {}
    reference[file.stem]["reference"]  = x_df["timestamp"].values
    
# gtzan
for file in gtzan.track_ids:
    reference[file] = {}
    reference[file]["reference"] = gtzan.track(file).beats.times

# gather estimations
estimations = experiments_path.glob("*")
for folder in estimations:
    print(f"Reading files from /{folder.name}")
    for file in folder.glob("*.npz"):
        reference[file.stem][folder.name] = np.load(file)["estimated"]

In [ ]:
beat_df = pd.DataFrame(reference).transpose()
beat_df.index.name = "track_id"
beat_df.to_csv(experiments_path / "estimation.csv")

In [ ]:
beat_df.info()

In [ ]:
beat_df = pd.read_csv(experiments_path / "estimation.csv")
beat_df = beat_df.set_index("track_id")

beat_df["reference"] = beat_df["reference"].to_numpy()
beat_df["madmom_rnn"] = beat_df["madmom_rnn"].to_numpy()
beat_df["madmom_tcn"] = beat_df["madmom_tcn"].to_numpy()
beat_df["librosa"] = beat_df["librosa"].to_numpy()
beat_df["essentia"] = beat_df["essentia"].to_numpy()

In [ ]:
beat_df.info()

In [ ]:
beat_df["madmom_rnn_metrics"] = beat_df[["reference", "madmom_rnn"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["madmom_rnn"]), axis=1)
beat_df["librosa_metrics"] = beat_df[["reference", "librosa"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["librosa"]), axis=1)
beat_df["madmom_tcn_metrics"] = beat_df[["reference", "madmom_tcn"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["madmom_tcn"]), axis=1)
beat_df["essentia_metrics"] = beat_df[["reference", "essentia"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["essentia"]), axis=1)

In [ ]:
def tmp(column_dict, index_column):
    column_dict["track_id"] = index_column
    return column_dict

In [ ]:
beat_df["madmom_rnn_metrics"].iloc[0]

In [ ]:
pd.json_normalize(beat_df["madmom_rnn_metrics"].reset_index().apply(lambda x: tmp(x["madmom_rnn_metrics"], x["track_id"]), axis=1))

In [ ]:
# gambiarra nossa de cada dia
madmom_rnn_metrics = pd.json_normalize(beat_df["madmom_rnn_metrics"].reset_index().apply(lambda x: tmp(x["madmom_rnn_metrics"], x["track_id"]), axis=1)).set_index("track_id")
madmom_tcn_metrics = pd.json_normalize(beat_df["madmom_tcn_metrics"].reset_index().apply(lambda x: tmp(x["madmom_tcn_metrics"], x["track_id"]), axis=1)).set_index("track_id")
librosa_metrics = pd.json_normalize(beat_df["librosa_metrics"].reset_index().apply(lambda x: tmp(x["librosa_metrics"], x["track_id"]), axis=1)).set_index("track_id")
essentia_metrics = pd.json_normalize(beat_df["essentia_metrics"].reset_index().apply(lambda x: tmp(x["essentia_metrics"], x["track_id"]), axis=1)).set_index("track_id")

In [ ]:
madmom_tcn_metrics

In [ ]:
madmom_tcn_metrics.join(dataset_df).groupby("dataset").median()

In [ ]:
madmom_rnn_metrics.join(dataset_df).groupby("dataset").median()

In [ ]:
essentia_metrics.join(dataset_df).groupby("dataset").median()

In [ ]:
librosa_metrics.join(dataset_df).groupby("dataset").median()